# Overview
This Jupyter notebook helps you create a 3D model using the Photoscan photogrammetry software, leveraging the computing power of a high-performance compute cluster. It assumes that your photographs are stored in a folder on [Box](http://berkeley.box.com), and the results of each step are saved back to Box.

If you want to manually edit the results of any step in the process, you can sync or download the project file and photos from Box to any computer with Photoscan installed (version 1.5 or higher), make your edits, and move the changes back to Box before proceeding with the next computationally-intensive processing step.

## Software versions
This notebook has been tested using:
* python 3.5 kernel
* boxsdk (2.0.0a2)
* Photoscan XXXXXXXX?

## Workflow at a glance
The photogrammetry workflow laid out here involves the following steps. Each listed step has its own section in the notebook. You can click the "+" icon next to each section to expand it, and from there you can configure and run the code.
1. **Project Setup**: defines where various source files are located.
2. **Box Setup**: connect this notebook to Box so it can access the photos you've stored there. 
3. **Align Photos**: arranges the photos you've taken, relative to one another. 1% to 8% of the expected processing time.
4. **Build Dense Cloud**: creates the a point cloud from the photographs. The most computationally-intensive step, which can take 50%-96% of the total model generation time, depending on your settings. This step is run using GPU nodes on the cluster, which improve performance.
5. **Build Mesh**: connecting the point cloud together to build a mesh takes around 30% of the processing time on lower quality settings, or as little as 2% on higher quality settings.
6. **Build Texture**: building the texture takes between 8% (medium quality) and 1% (ultra high quality) of the processing time.

# Project Setup
This step connects the notebook to Box. Before you get started, make sure you have a folder in Box for your project. That folder should have a folder inside it called "images" that has all the photos you want to use for your 3D model. This workflow will also create a new folder, "files", that contains the project files-- you don't have to set that up in advance.

Click in the first code box below, and then click the "run cell" button in the toolbar. This will run the code and move to the next box. Repeat until the next set of instructions.

In [21]:
# widget imports
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from IPython.display import display


In [22]:
# Create text widget for projectname
projectname = widgets.Text()

# Create text widget for input
input_text = widgets.Text()


def setprojectname(projectname):
    return projectname


In [23]:
# Define function to bind value of the input to the output variable 
def bind_input_to_projectname(sender):
    projectname.value = input_text.value
    
input_text.on_submit(bind_input_to_projectname) 

## Input folder name
After you run the code box below, a text box will appear. Click in the text box and type in the name of the folder on Box where you've uploaded your images, then hit the "run cell" button again.

In [24]:
# Enter the projectname here:
# Display input text box widget for input
input_text

# Display projectname text box widget (will populate when value submitted in input)
projectname

## Project set-up
The code box below defines the locations of a number of things for your project. The defaults are usually fine, but depending on your compute environment, you may need to change some of these.
    
By default:
* The processing jobs will run in /global/scratch/[your-user-name]/[your-project-name]
* It's assuming your home folder is in /global/home/users/[your-user-name]
* The Singularity container (a particular way of packaging up software so it can run in an HPC environment) that has Photoscan in it should be in /global/scratch/mmanning/photoscan13.img **(FIX THIS!!!)**

After you run the next code box, text will appear confirming these paths. If you need to make any changes, edit the code box and run it again.

In [25]:
# Defines all paths and filenames
import os

usernamearray = %sx whoami
username = usernamearray[0]
print('user name: ', username)
print('project name: ', projectname.value)

#Modify the run folder location here
runFolder = '/global/scratch/' + username + '/'  + projectname.value + '/'

#Modify the home folder location here
homeFolder = '/global/home/users/' + username + '/' 
print('run folder: ', runFolder)

# Creates the run folder if it does not exist
if ( not os.path.isdir(runFolder)):
    print('run folder does not exist, creating.')
    os.mkdir(runFolder)

singularitymountfolder = '/scratch/'

boxProjectFolder = projectname.value
projectFile = projectname.value + '.psx'
datazipFile = projectname.value + '.files.zip'
dataFolder = projectname.value + '.files'

projectfileid = ''
objFile = projectname.value + '.obj'
jpgFile = projectname.value + '.jpg'
orthoFile = projectname.value + '.tiff'

scratchImageDataDirectory = runFolder + 'images/'
# Creates the image folder if it does not exist
if ( not os.path.isdir(scratchImageDataDirectory)):
    print('images folder does not exist, creating.')
    os.mkdir(scratchImageDataDirectory)

slurmScript = runFolder + 'slurmscript.sh'
execScript = runFolder + 'execscript.sh'
scratchExecScript = singularitymountfolder + 'execscript.sh'
commandScript = runFolder + 'commandscript.sh'

#Modify the path for the Singularity container here
singularityContainerPath = '/global/scratch/mmanning/photoscan13.img'

user name:  quinnd
project name:  
run folder:  /global/scratch/quinnd//


# Box Setup

To connect Box with this notebook, you need to go through some one-time configuration steps. If you run this notebook again to create a 3D model out of a new set of photos, you won't have to run this step again. First, run the code below.

In [26]:
def store_tokens(access_token, refresh_token):
    
    """Callback for storing refresh tokens. (For now we ignore access tokens)."""
    with open(homeFolder + 'apptoken.cfg', 'w') as f:
     f.write(refresh_token.strip())

## Creating a Box application

To get access to the Box API (programmatic access to Box that you can use with this notebook), you have to register an "application" via the Box Developers site. 

For Berkeley, the Box Developers site is at: https://berkeley.app.box.com/developers/services/edit/. For other institutions, append */developers/console/newapp* to the URL when you're logged in with your institutional credentials.

Once you're at the Box Developers site:

1. Click *Enterprise Integration*, then click the *Next* button.
2. On the next page, choose *Standard OAuth 2.0 (User Authentication)*
3. On the next page, enter a name for this application, such as "Photoscan notebook" and click the *Create App* button.
4. Click the *View your app* button.
5. The next page shows a number of fields. The ones you need are *Client ID*, *Client secret*, and *Redirect URI*. Copy each of those and paste them into the corresponding place in the code box below, then run the code box.

**FIGURE OUT WHY THIS DOESN'T WORK**

-----
Oauth2 information is read from a local file with three lines, one line per parameter. 
The client id and client secret are defined in the Box application created for this notebook.  Create the application at the Box Developers site: https://berkeley.app.box.com/developers/services/edit/

The redirect uri can be any site that requires validation. Run the bootstrap notebook to create initial 
tokens that are then continually refreshed. **What's an example so we don't make people look for one?**

In [27]:
import os

CLIENT_ID = None
CLIENT_SECRET = None
REDIRECT_URI = None
# Read app info from text file
with open(homeFolder + 'app.cfg', 'r') as app_cfg:
    CLIENT_ID = app_cfg.readline()
    CLIENT_SECRET = app_cfg.readline()
    REDIRECT_URI = app_cfg.readline()


FileNotFoundError: [Errno 2] No such file or directory: '/global/home/users/quinnd/app.cfg'

The refresh token is read from a local file.
This token was created by running the bootstrap notebook which requires the user to validate
with CalNet Authentication Service credentials, then stores the returned auth and refresh tokens 
in the same config files.

In [ ]:
REFRESH_TOKEN = None

# Read app info from text file
with open(homeFolder + 'apptoken.cfg', 'r') as apptoken_cfg:
    REFRESH_TOKEN = apptoken_cfg.readline()

__Perform autentication__ then create globus client
Verify client is working by retrieving the name of the users root folder in Box

In [ ]:
from boxsdk import OAuth2
from boxsdk import Client

# Do OAuth2 authorization.
oauth = OAuth2(
    client_id=CLIENT_ID.strip(),
    client_secret=CLIENT_SECRET.strip(),
    refresh_token=REFRESH_TOKEN.strip(),
    store_tokens=store_tokens
)

client = Client(oauth)

root_folder = client.folder(folder_id='0').get()
print ("folder name: ", root_folder['name'] )

items = client.folder(folder_id='0').get_items(limit=100, offset=0)
#print ("items: ", items )



### Utility functions
**Do you have to run all of these before using the notebook for photogrammetry?**

__function to find folder id be folder name.__
Current SDK does not have a 'find by name' function so must loop thru all folders and look for match.

In [ ]:
def find_folder_id(folder_name):
    print ('find_folder_id folder_name: ', folder_name)
    folderlist = client.search(query=folder_name, result_type='folder', limit=100, offset=0)
    
    print ('find_folder_id folderlist: ', folderlist)
    
    if len(folderlist) == 0:
        print('folder not found: ', folder_name)
        return 0
    else:
        for fldr in folderlist:
            if fldr['name'] == folder_name :
                return fldr['id']
            
        return 0

__Create command scripts template.__
Write out a execute file that includes the license install as the first step each time to enable any HPC node.
This is a temporary workaround until release 1.3 is installed on Savio and a license server is configured.
execscript - contains the license install as the first command and the command script as the second command.

In [ ]:
# script for singularity to run
import os, stat

execScriptTemplate = '/opt/photoscan-pro/photoscan.sh --activate TGN25-21RGK-UM9NG-UK49O-V55ZO\n\
/opt/photoscan-pro/photoscan.sh -r  /scratch/commandscript.sh \n'

print (' exec script output: ', execScriptTemplate )

with open(execScript, 'w') as f:
    f.write(execScriptTemplate)
    
os.chmod(execScript,  stat.S_IRWXU)    





In [ ]:
# batch script
batchtemplate = '#!/bin/bash  \n\
# Job name: \n\
#SBATCH --job-name=' + projectname.value + '\n\
# \n\
# Account: \n\
#SBATCH --account=ac_scsguest \n\
# \n\
# Partition: \n\
#SBATCH --partition=savio \n\
# \n\
# Wall clock limit: \n\
#SBATCH --time={} \n\
# \n\
## Command(s) to run: \n\
singularity exec -B ' + runFolder + ':' + singularitymountfolder + '  ' + singularityContainerPath + '  ' + scratchExecScript + '\n' 
  
    

__Update the project file in the Box folder.__ 
Multiple times in the workflow we need to move the project file back to Box for manual editing on the desktop 
then retrieve the updated the file and continue the workflow. This utility funcation pushes the project 
file into the working directory on Savio. It assumes that the project folder will only contain one project file and one zipped archive.

In [ ]:
import os
import shutil

def update_project_file_in_box():

    newFolderId = find_folder_id(boxProjectFolder)
    print ("Box folder id: ", newFolderId  )
    
    # create a zipped archive of the data folder
    shutil.make_archive(runFolder + dataFolder, 'zip', runFolder + dataFolder)
    
    projectfilelist = client.search(query=projectFile, result_type='file', limit=10, 
                                offset=0, ancestor_folders=[client.folder(folder_id=newFolderId)],
                                file_extensions=['psx'] )  
    
    print ("project file list: ", projectfilelist  )
    
    # if project file is not yet in folder, upload both the project file and the data zip file
    if len(projectfilelist) == 0 or len(projectfilelist) > 1: 
        print ("project file not found. " )
        upload_folder = client.folder(folder_id=newFolderId).get()
        projectpsx = upload_folder.upload(runFolder + projectFile)  
        print ("project file id: ", projectpsx['id'] )
        projectzip = upload_folder.upload(runFolder + datazipFile)  
        print ("data zip file id: ", projectzip['id'] )
        return
    else:
        datazipfilelist = client.search(query=datazipFile, result_type='file', limit=10, 
                                offset=0, ancestor_folders=[client.folder(folder_id=newFolderId)])
                                #file_extensions=['zip'] )  
        print ("datazip file list: ", datazipfilelist  )
        
        projectfileid = projectfilelist[0]['id']
        print ("project file id: ", projectfileid )
        datazipfileid = datazipfilelist[0]['id']
        print ("data zip file list: ", datazipfileid  )
    
        update_file = client.file(file_id=projectfileid).get()
        update_zip_file = client.file(file_id=datazipfileid).get()

        # upload a new version of the project file
        print ('begin project file update.' )
        psxfile = update_file.update_contents(runFolder + projectFile)  
        zipfile = update_zip_file.update_contents(runFolder + datazipFile)
        print ('update psx result: ', psxfile ,'   update zip result: ', zipfile )

__Retrieve the project file from the Box folder.__ 
Multiple times in the workflow we need to move the project file back to Box for manual editing on the desktop 
then retrieve the updated the file and continue the workflow. This utility funcation pulls the updated project 
file into the working directory on Savio.

In [ ]:

import os
import shutil 
def retrieve_project_file():
    newFolderId = find_folder_id(boxProjectFolder)
    tgtitems = client.folder(folder_id=newFolderId).get_items(limit=1000, offset=0)

    # download the project file
    for tgtitem in tgtitems:
        if  not tgtitem['type'] == 'folder' and tgtitem['name'].endswith('.psx'):
            print('downloading: ', tgtitem['name'])
            imagecontent = client.file(file_id=tgtitem['id']).content()
            newfile = open(runFolder + tgtitem['name'], 'wb')
            newfile.write(imagecontent)
            newfile.close()
            print('project file download complete.')
            
        if  not tgtitem['type'] == 'folder' and tgtitem['name'].endswith('.zip'):
            
            # delete the old folder
            !rm -rf $runFolder$dataFolder
            
            print('downloading: ', tgtitem['name'])
            imagecontent = client.file(file_id=tgtitem['id']).content()
            newfile = open(runFolder + tgtitem['name'], 'wb')
            newfile.write(imagecontent)
            newfile.close()
            
            shutil.unpack_archive(runFolder + datazipFile, runFolder + dataFolder ,'zip')
            
            print('data zip file download complete.')

### Retrieve the images from the Box folder.
currently the Box SDK does not have an option for finding a folder by name so if you are looking for a specific folder then you would need to loop thru all the items in the list below and do a name match. Once you find the folder and retrieve the id, you can save that id for subsequent runs. Another option is to get the id from the url in the web client, but approah below is more flexible for now. 

In [ ]:
import os
import shutil 

print ('current working directory: ', os.getcwd())
os.chdir(scratchImageDataDirectory)

# test folder contents
items = client.folder(folder_id='0').get_items(limit=20, offset=0)
if type(items) is list:
    print ('number of files in top folder: ', len(items) )
    
    targetfolderId = ''
    for item in items:
        if item['type'] == 'folder':
            print('folder name: ', item['name'])
            if item['name'] == boxProjectFolder:
                targetfolderId = item['id']
                print('targetfolderId: ', targetfolderId)
        
    if targetfolderId is not None:
        tgtitems = client.folder(folder_id=targetfolderId).get_items(limit=200, offset=0)
        if type(tgtitems) is list:
            print ('number of files in target folder: ', len(tgtitems) ) 
        
        # download all image files
        for tgtitem in tgtitems:
            if  not tgtitem['type'] == 'folder' and tgtitem['name'].endswith('.JPG'):
                print('dowmnloading: ', tgtitem['name'])
                imagecontent = client.file(file_id=tgtitem['id']).content()
                newfile = open(scratchImageDataDirectory + tgtitem['name'], 'wb')
                newfile.write(imagecontent)
                newfile.close()

### Load the image set
run the first photoscan step: add the files
this cell created the script that will be executed by photoshop by writes the necessary commands into a text file.
mod the text in the step1Template string below as required.

clear docs about what needs to be set, can params be highlighted?


In [ ]:
import os, stat

files = [ singularitymountfolder + 'images/' + f for f in os.listdir(scratchImageDataDirectory) ] 

template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
import time \n\
doc = PhotoScan.app.document \n\
chunk = PhotoScan.app.document.addChunk() \n\
chunk.addPhotos( {} ) \n\
doc.save(path=\"{}\", chunks = [doc.chunk])\n '

output = template.format(str(files), singularitymountfolder + projectFile)

with open(commandScript, 'w') as f:  
    f.write(output)

os.chmod(commandScript,  stat.S_IRWXU)

#set time limit for this batch run
outputbatchscript = batchtemplate.format('00:30:00')
with open(slurmScript, 'w') as f:  
    f.write(outputbatchscript)

#### Execute the image load

In [ ]:
os.chdir(runFolder)

out = !sbatch slurmscript.sh   
    
print ('Execute the image load output: ', out ) 


In [ ]:
!squeue -u $username

!scontrol show job 1081527

__Create a new folder in the base directory and upload the project (.psx and data zip) file in the current folder.__
If the folder for the project does not exisit, the folder is created.
If the folder already exists and contains a project file of the same name, the project file created in the previous step is NOT uploaded.
#### <span style="color:red">New folders in Box can take several minutes to 'register'</span>

In [ ]:
from boxsdk.exception import BoxAPIException

newFolderId = find_folder_id(boxProjectFolder)
print('folder id: ', newFolderId)

if newFolderId == 0:
    newFolder = client.folder(folder_id='0').create_subfolder(boxProjectFolder)
    newFolderId = newFolder['id']
    print ("folder created" )
else: 
    print ("folder exists" )
    
print ("folder id: ", newFolderId )

update_project_file_in_box()

#### <span style="color:red">OFFLINE STEP:</span> User masks images in the project (psx) file locally using Photoscan on desktop.

# Build Dense Cloud

Begin by retrieving the project file, by running the code box below.

In [ ]:
retrieve_project_file()

## Configuring and setting up job script

Run the code box below, then select your depth filtering mode and dense point cloud quality from the drop-downs. 

Click the "run code" button again, then run the subsequent code boxes.

-----
Create the script for the second step: dense cloud build
this cell created the script that will be executed by photoshop by writes the necessary commands into a text file.
mod the text in the step2Template string below as required.

Options for PhotoScan.FilterMode: Depth filtering mode in [NoFiltering, MildFiltering, ModerateFiltering, AggressiveFiltering]

Options for PhotoScan.Quality: Dense point cloud quality in [UltraQuality, HighQuality, MediumQuality, LowQuality, LowestQuality]

In [ ]:
## DO SOME WIDGET STUFF HERE WITH SELECTION WIDGET, E.G.
widgets.Select(
    options=['Linux', 'Windows', 'OSX'],
    value='OSX',
    # rows=10,
    description='OS:',
    disabled=False
)

In [ ]:
# Creates job script for building dense cloud
import os, stat

template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
import time \n\
print( "start time: ", time.time()) \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.matchPhotos(accuracy=PhotoScan.HighAccuracy, preselection=PhotoScan.GenericPreselection) \n\
chunk.alignCameras(adaptive_fitting=True) \n\
chunk.buildDenseCloud(quality=PhotoScan.MediumQuality,  filter=PhotoScan.ModerateFiltering  ) \n\
doc.save(\"{}\") \n\
print( "stop time: ", time.time()) \n'

output = template.format(singularitymountfolder + projectFile, singularitymountfolder + projectFile)

with open(commandScript, 'w') as f:  
    f.write(output)

    
#Set time limit for this batch run
outputbatchscript = batchtemplate.format('01:30:00')
with open(slurmScript, 'w') as f:  
    f.write(outputbatchscript)
    

## Running job script
Running the following code box will submit your job to the cluster. Your job ID will be printed below.

In [ ]:
out = !sbatch slurmscript.sh   
    
print ('Execute the dense cloud build output: ', out ) 
jobId =  out[0].split()[3]
print (jobId)

## Checking on job status
Run the following code box to check on the status of the dense cloud build. You can run it multiple times for long-running jobs to get new updates.

In [ ]:
# print the users queue and the job status by id
!squeue -u $username
print('--------------------------------')
!scontrol show job $jobId

## What is this?

In [ ]:
# print the log file if necessary
slurmlogfilename = 'slurm-' + jobId + '.out'
with open(slurmlogfilename, 'r') as fin:
    print(fin.read(), end="")

## Export results
**What happens here??**

In [ ]:
template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.exportPoints(path=\"{}\", format=PhotoScan.PointsFormat.PointsFormatOC3)  \n' 

output = template.format(singularitymountfolder + projectFile,  singularitymountfolder + 'photoscandemo.oc3')

with open(commandScript, 'w') as f:
    f.write(output)

In [ ]:
out = !sbatch slurmscript.sh   
        
print (' Export the points  output: ', out ) 
jobId =  out[0].split()[3]
print (jobId)

## Move the updated project file back to Box
After generating the dense cloud, this will be a large file.
** Is there some sort of success message?**

In [ ]:
update_project_file_in_box()

## Follow-up steps
At this point, you may want to clean up the dense point cloud by removing stray points (e.g. for objects that have not been photographed in a lightbox, where there might be artifacts from the background, etc.) This should be done using the desktop version of Photoscan. 

Be sure to save the updated project file back to the same place in Box if you want to run the next processing steps through this notebook.

**Stick in something about AEoD**

# Build the mesh

__Retrieve the project file (psx) from Box as it may have been updated.__

In [ ]:
retrieve_project_file()

__build the mesh__

Surface type in [Arbitrary, HeightField]

Face count in [LowFaceCount, MediumFaceCount, HighFaceCount]

Data source in [PointCloudData, DenseCloudData, ModelData, TiledModelData, ElevationData, OrthomosaicData]

In [ ]:
template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
import time \n\
print( "start time: ", time.time()) \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.buildModel(surface=PhotoScan.Arbitrary, source=PhotoScan.DenseCloudData, face_count=PhotoScan.HighFaceCount ) \n\
doc.save(\"{}\") \n\
print( "stop time: ", time.time()) \n'

output = template.format(singularitymountfolder + projectFile, singularitymountfolder + projectFile)

with open(commandScript, 'w') as f:  
    f.write(output)


#set time limit for this batch run
outputbatchscript = batchtemplate.format('00:30:00')
with open(slurmScript, 'w') as f:  
    f.write(outputbatchscript)
    

#### Execute the mesh build

In [ ]:
out = !sbatch slurmscript.sh   
    
print ('Execute the mesh build output: ', out ) 
jobId =  out[0].split()[3]
print (jobId) 

In [ ]:
# print the users queue and the job status by id
!squeue -u $username
print('--------------------------------')
!scontrol show job $jobId

In [ ]:
# print the log file if necessary
slurmlogfilename = 'slurm-' + jobId + '.out'
with open(slurmlogfilename, 'r') as fin:
    print(fin.read(), end="")

#### Move the resulting psx back to Box. 

In [ ]:
update_project_file_in_box()

#### <span style="color:red">OFFLINE STEP:</span> User adds marker points in the project (psx) file locally using Photoscan on desktop.

### Build the texture and the orthomosaic

__Retrieve the project file (psx) from Box as it may have been updated.__

In [ ]:
retrieve_project_file()

__build the texture and the orthomosaic__


Data source in [PointCloudData, DenseCloudData, ModelData, TiledModelData, ElevationData, OrthomosaicData]

To specify coordinate system add: 
chunk.crs = PhotoScan.CoordinateSystem("????")  \n\

In [ ]:
template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
import time \n\
print( "start time: ", time.time()) \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.buildUV(mapping=PhotoScan.GenericMapping) \n\
chunk.buildTexture( color_correction=True ) \n\
doc.save(\"{}\") \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.buildOrthomosaic( surface=PhotoScan.DataSource.ModelData ) \n\
doc.save(\"{}\") \n\
print( "stop time: ", time.time()) \n'

output = template.format(singularitymountfolder + projectFile, singularitymountfolder + projectFile, singularitymountfolder + projectFile, singularitymountfolder + projectFile)

with open(commandScript, 'w') as f:  
    f.write(output)


#### Execute the texture and orthomosaic build

In [ ]:
out = !sbatch slurmscript.sh   
    
print ('  texture and orthomosaic build output: ', out ) 
jobId =  out[0].split()[3]
print (jobId)    

#### Move the resulting psx back to Box. 

In [ ]:
update_project_file_in_box()

### Export results to Box

#### Export the orthomosaic format

In [ ]:
template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.exportOrthomosaic(path=\"{}\", image_format=PhotoScan.ImageFormat.ImageFormatTIFF,)  \n' 

output = template.format(singularitymountfolder + projectFile,  singularitymountfolder + orthoFile)

with open(commandScript, 'w') as f:
    f.write(output)


In [ ]:
out = !sbatch slurmscript.sh   
        
print (' Export the orthomosaic format output: ', out ) 
jobId =  out[0].split()[3]
print (jobId)  

#### Move the resulting Orthomosaic file to Box. 

In [ ]:
newFolderId = find_folder_id(boxProjectFolder)
upload_folder = client.folder(folder_id=newFolderId).get()
orthoUploaded = upload_folder.upload( runFolder + orthoFile)  
print ("orthomosaic file id: ", orthoUploaded['id'] )

#### Generate OBJ format and load to Box
then download and view at: http://3dviewer.net/

In [ ]:
template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.exportModel(path=\"{}\", format=PhotoScan.ModelFormat.ModelFormatOBJ)  \n' 

output = template.format(singularitymountfolder + projectFile, singularitymountfolder + objFile)

with open(commandScript, 'w') as f:
    f.write(output)


In [ ]:
out = !sbatch slurmscript.sh   
        
print (' Generate OBJ format output: ', out ) 
jobId =  out[0].split()[3]
print (jobId)         


#### Move the resulting OBJ file and JPEG file to Box.

In [ ]:
newFolderId = find_folder_id(boxProjectFolder)
upload_folder = client.folder(folder_id=newFolderId).get()
objUploaded = upload_folder.upload(runFolder + objFile)  
print ("obj file id: ", objUploaded['id'] )
jpgUploaded = upload_folder.upload(runFolder + jpgFile)  
print ("jpeg file id: ", jpgUploaded['id'] )

## License
This software is available under the terms of the Educational Community License, Version 2.0 (ECL 2.0). This software is Copyright 2016 The Regents of the University of California, Berkeley ("Berkeley").

The text of the ECL license is reproduced below.

Educational Community License, Version 2.0
*************************************
Copyright 2016 The Regents of the University of California, Berkeley ("Berkeley")

Educational Community License, Version 2.0, April 2007

The Educational Community License version 2.0 ("ECL") consists of the
Apache 2.0 license, modified to change the scope of the patent grant in
section 3 to be specific to the needs of the education communities using
this license. The original Apache 2.0 license can be found at:[http://www.apache.org/licenses/LICENSE-2.0]